# CNN Model

This model is an extension of the Base CNN model to incorporate data preprocessing, augmentation and regularization.

### Imports

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from utils import split_dataset, train_model, plot_metrics, save_model, load_model

### Initialize Dataset

In [ ]:
# Constants
TRAIN_DATA_DIR = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train'
SAVE_MODEL_FILE = 'models/cnn.pt'
SEED = 0
NUM_SAMPLES = 1000
TRAIN_SPLIT = 0.80
BATCH_SIZE = 100
LEARN_RATE = 5e-4
EPOCHS = 10

In [ ]:
# Seed PyTorch
torch.manual_seed(SEED)

# Image transforms
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((100, 100))
])

# Initialize dataset
dataset = torchvision.datasets.ImageFolder(
    root=TRAIN_DATA_DIR,
    transform=transforms
)

# Split dataset into training and validation sets
train_loader, valid_loader = split_dataset(dataset, NUM_SAMPLES, TRAIN_SPLIT, BATCH_SIZE, SEED)

# Check for CUDA GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

In [ ]:
# Convolutional neural network
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 32, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        # self.dropout1 = nn.Dropout2d(0.2)

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        # self.dropout2 = nn.Dropout2d(0.2)

        # Fully connected layer 1
        self.fc1 = nn.Linear(32 * 25**2, 128)
        # self.dropout3 = nn.Dropout2d(0.2)

        # Fully connected layer 2
        self.fc2 = nn.Linear(128, 64)
        # self.dropout4 = nn.Dropout2d(0.2)

        # Output layer
        self.fc3 = nn.Linear(64, 29)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate model and move to GPU if available
model = CNN()
model = model.to(device)

# Print model details
from torchsummary import summary
summary(model, input_size=(3, 100, 100))

### Training

In [ ]:
metrics = train_model(model, train_loader, valid_loader, LEARN_RATE, EPOCHS, device, conv=True)

In [ ]:
save_model(SAVE_MODEL_FILE, model, metrics)

### Evaluation

In [ ]:
# metrics = load_model(SAVE_MODEL_FILE, model)

In [ ]:
plot_metrics(metrics)